In [187]:
%pylab inline
from snakebite.client import Client
import pyaml as yaml
import sys
import pythonrun
reload(pythonrun)
import os
import seaborn as sns
from sklearn import metrics
import scala_python_endive_wrapper
reload(scala_python_endive_wrapper)
from scala_python_endive_wrapper import *
import pandas as pd
import os
from sklearn.cluster import KMeans
import itertools

Populating the interactive namespace from numpy and matplotlib


In [2]:
hdfsclient = Client("amp-spark-master.amp", 8020, use_trash=False)
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')
logpath = "/tmp/log"

In [3]:
executor_mem = "200g"
cores_per_executor = 32
num_executors = 14
EGR1_PATH = '/user/vaishaal/endive-data/EGR1'

In [4]:
make_dnase_gaussian_filter_gen(1.0, 1.0)(100).shape

(100, 232)

In [194]:
# this is where the magic happens
results = [] 
num_filters = [256]
gammas = [1.0]
dnase_gammas = [1.0]
kmer_sizes = [8]
dnase_kmer_sizes = [100]
replicates = [0,1]

params = list(itertools.product(num_filters, gammas, dnase_gammas, kmer_sizes, dnase_kmer_sizes, replicates))

In [195]:
params

[(256, 1.0, 1.0, 8, 100, 0), (256, 1.0, 1.0, 8, 100, 1)]

In [185]:
chromosomes = map(lambda x: "chr{0}".format(x+1), range(23))
# CHANGE ME BASED ON ENUM + TF
egr1_cell_types = string_to_enum_celltypes(['GM12878', 'H1hESC', 'HCT116', 'MCF7'])
for param in params:
    num_filter, gamma, dnase_gamma, kmer_size, dnase_kmer_size, replicate = param
    feat_out_loc = "/user/vaishaal/featurizedWindows/sequence/tiny/{0}-filters/{1}-gamma/{2}-dnase-gamma/{3}-kmers/{4}-dnase-kmers/replicate-{5}"\
                    .format(num_filter, gamma, dnase_gamma, kmer_size, dnase_kmer_size, replicate)
    res = run_kitchensink_featurize_pipeline(EGR1_PATH,
                           "/tmp/filters.csv", 
                           logpath, 
                           seed=0,
                           cores_per_executor=cores_per_executor,
                           sample=0.01,
                           alphabet_size=4,
                           kmer_size=kmer_size,
                           dnase_kmer_size=dnase_kmer_size,
                           gamma=gamma,
                           dnase_gamma=dnase_gamma,
                           executor_mem=executor_mem,
                           num_filters=num_filter,
                           num_partitions=cores_per_executor*num_executors,
                           featuresOutput=feat_out_loc,
                           num_executors=num_executors)
                

W SHAPE  (256, 232)
{'kmerLength': 8, 'reference': '/home/eecs/akmorrow/ADAM/endive/workfiles/hg19.2bit', 'filtersPath': '/tmp/filters.csv', 'numPartitions': 448, 'dnaseBams': '/data/anv/DREAMDATA/dnase_bams/merged_coverage/', 'approxDim': 256, 'aggregatedSequenceOutput': '/user/vaishaal/endive-data/EGR1', 'featuresOutput': '/user/vaishaal/featurizedWindows/sequence/tiny/256-filters/1.0-gamma/1.0-dnase-gamma/8-kmers/100-dnase-kmers/replicate-0', 'seed': 0, 'dnaseNarrow': '/data/anv/DREAMDATA/DNASE/peaks/relaxed/', 'dnaseKmerLength': 100, 'featurizeSample': 0.01, 'alphabetSize': 4, 'readFiltersFromDisk': True}
W SHAPE  (256, 232)
{'kmerLength': 8, 'reference': '/home/eecs/akmorrow/ADAM/endive/workfiles/hg19.2bit', 'filtersPath': '/tmp/filters.csv', 'numPartitions': 448, 'dnaseBams': '/data/anv/DREAMDATA/dnase_bams/merged_coverage/', 'approxDim': 256, 'aggregatedSequenceOutput': '/user/vaishaal/endive-data/EGR1', 'featuresOutput': '/user/vaishaal/featurizedWindows/sequence/tiny/256-filte

In [ ]:
for param in params:
    num_filter, gamma, dnase_gamma, kmer_size, dnase_kmer_size, replicate = param
    feat_out_loc = "/user/vaishaal/featurizedWindows/sequence/tiny/{0}-filters/{1}-gamma/{2}-dnase-gamma/{3}-kmers/{4}-dnase-kmers/replicate-{5}"\
    .format(num_filter, gamma, dnase_gamma, kmer_size, dnase_kmer_size, replicate)
    print("FEAT OUT LOC " + feat_out_loc)
    meta = {}
    meta['num_filter'] = num_filter
    meta['dnase_kmer_size'] = dnase_kmer_size
    meta['dnase_gamma'] = dnase_gamma
    meta['gamma'] = gamma
    results_df = cross_validate(feat_out_loc, 
                            hdfsclient, 
                            chromosomes, 
                            egr1_cell_types,
                            numHoldOutChr=1, 
                            numHoldOutCell=1,
                            cores_per_executor = 32,
                            num_executors = 14,
                            executor_mem="150g",
                            num_folds=1, 
                            negativeSamplingFreqs=[0.001],
                            regs=[1.0, 1.0],
                            other_meta=meta)
    print(results_df)
    results_dfs.append(results_df)
            
            


    

FEAT OUT LOC /user/vaishaal/featurizedWindows/sequence/tiny/256-filters/1.0-gamma/1.0-dnase-gamma/8-kmers/100-dnase-kmers/replicate-0
FOLD 0
HOLDING OUT CHROMOSOMES ['chr12']
HOLDING OUT CELL TYPES [3]
RUNING SOLVER WITH REG=1.0


In [163]:
print(results_df.to_csv())

,dnase_gamma,dnase_kmer_size,gamma,negativeSamplingFreq,num_filter,reg,test_celltypes,test_chromosomes,train_auPRC,train_auROC,train_recall_at_05_fdr,train_recall_at_10_fdr,train_recall_at_25_fdr,train_recall_at_50_fdr,val_auPRC,val_auROC,val_recall_at_05_fdr,val_recall_at_10_fdr,val_recall_at_25_fdr,val_recall_at_50_fdr
0,1.0,100,1.0,0.001,256,1.0,[3],['chr12'],0.156625337867,0.986289722268,0.0,0.0,0.0,0.0,0.0766458060501,0.982864743542,0.0,0.0,0.0,0.0
1,1.0,100,1.0,0.001,256,1.0,[3],['chr12'],0.162404536922,0.986410500489,0.0,0.0,0.0,0.0,0.0854588690543,0.984974022807,0.0,0.0,0.0,0.0
2,1.0,100,1.0,0.001,256,1.0,[3],['chr12'],0.157697639049,0.986612763443,0.0,0.0,0.0,0.0,0.0887482579539,0.987081739065,0.0,0.0,0.0,0.0833333333333
3,1.0,100,1.0,0.001,256,1.0,[3],['chr12'],0.152685088849,0.986184338905,0.0,0.0,0.0,0.0,0.0653691291953,0.984803654938,0.0,0.0,0.0,0.0
4,1.0,100,1.0,0.001,256,1.0,[3],['chr12'],0.16243771816,0.986531446019,0.0,0.0,0.0,0.0,0.0807973169962,0.980825799635,0.0,0.0

In [181]:
pd.concat(results_dfs)

,dnase_gamma,dnase_kmer_size,gamma,negativeSamplingFreq,num_filter,reg,test_celltypes,test_chromosomes,train_auPRC,train_auROC,train_recall_at_05_fdr,train_recall_at_10_fdr,train_recall_at_25_fdr,train_recall_at_50_fdr,val_auPRC,val_auROC,val_recall_at_05_fdr,val_recall_at_10_fdr,val_recall_at_25_fdr,val_recall_at_50_fdr
0,1,100,1,0.001,1024,1,[3],[chr12],0.500993,0.500000,0,0,0,0,0.500448,0.500000,0,0,0,0.000000
1,1,100,1,0.001,1024,1,[3],[chr12],0.500993,0.500000,0,0,0,0,0.500448,0.500000,0,0,0,0.000000
2,1,100,1,0.001,1024,1,[3],[chr12],0.500993,0.500000,0,0,0,0,0.500448,0.500000,0,0,0,0.000000
3,1,100,1,0.001,1024,1,[3],[chr12],0.500993,0.500000,0,0,0,0,0.500448,0.500000,0,0,0,0.000000
4,1,100,1,0.001,1024,1,[3],[chr12],0.500993,0.500000,0,0,0,0,0.500448,0.500000,0,0,0,0.000000
0,1,100,1,0.001,256,1,[3],[chr12],0.157763,0.986390,0,0,0,0,0.086075,0.983270,0,0,0,0.041667
1,1,100,1,0.001,256,1,[3],[chr12],0.161938,0.986485,0,0,0,0,0.077896,0.985528,0,0,0,0.000000
0,1,100,1,0.001,256,1,[3],[chr12],0.149742,0.986034,0,0,0,0,0.068203,0.985679,0,0,0,0.000000
1,1,100,1,0.001,256,1,[3],[chr12],0.161915,0.986324,0,0,0,0,0.084936,0.981727,0,0,0,0.000000
0,1,100,1,0.001,256,1,[3],[chr12],0.148191,0.986077,0,0,0,0,0.060396,0.986467,0,0,0,0.000000


In [112]:
3results_df

,negativeSamplingFreq,reg,test_celltypes,test_chromosomes,train_auPRC,train_auROC,train_recall_at_05_fdr,train_recall_at_10_fdr,train_recall_at_25_fdr,train_recall_at_50_fdr,val_auPRC,val_auROC,val_recall_at_05_fdr,val_recall_at_10_fdr,val_recall_at_25_fdr,val_recall_at_50_fdr
0,0.01,1000,[3],[chr12],0.358819,0.958541,0.018828,0.026998,0.065364,0.291297,0.148956,0.925958,0.074074,0.074074,0.074074,0.074074


0.88036847526310447

In [35]:
feat_out_loc

'/user/vaishaal/featurizedWindows/sequence/tiny/256-filters/1.0-gamma/1.0-dnase-gamma/8-kmers/100-dnase-kmers/replicate-0'

In [61]:
old_result_dfs = pd.concat(results_dfs)

In [138]:

old_result_dfs

,dnase_gamma,dnase_kmer_size,gamma,negativeSamplingFreq,reg,test_celltypes,test_chromosomes,train_auPRC,train_auROC,train_recall_at_05_fdr,train_recall_at_10_fdr,train_recall_at_25_fdr,train_recall_at_50_fdr,val_auPRC,val_auROC,val_recall_at_05_fdr,val_recall_at_10_fdr,val_recall_at_25_fdr,val_recall_at_50_fdr
0,1,8,0.1,1,1.000000e-08,[3],[chr12],0.112224,0.858721,0.007273,0.007273,0.038182,0.089091,0.046896,0.964402,0,0,0,0
0,1,16,0.1,1,1.000000e-08,[3],[chr12],0.096167,0.876597,0.000000,0.000000,0.000000,0.042808,0.019243,0.789216,0,0,0,0
0,1,32,0.1,1,1.000000e-08,[3],[chr12],0.051959,0.871315,0.000000,0.000000,0.000000,0.000000,0.028478,0.970516,0,0,0,0
0,1,8,0.1,1,1.000000e-08,[3],[chr12],0.126049,0.867352,0.001724,0.001724,0.043103,0.112069,0.001489,0.733996,0,0,0,0
0,1,16,0.1,1,1.000000e-08,[3],[chr12],0.060356,0.873831,0.000000,0.000000,0.000000,0.003552,0.066907,0.827730,0,0,0,0
0,1,32,0.1,1,1.000000e-08,[3],[chr12],0.072030,0.877629,0.000000,0.000000,0.000000,0.000000,0.015330,0.504122,0,0,0,0
0,1,8,0.1,1,1.000000e-08,[3],[chr12],0.122803,0.863330,0.003490,0.003490,0.031414,0.097731,0.004694,0.899637,0,0,0,0
0,1,16,0.1,1,1.000000e-08,[3],[chr12],0.078510,0.870935,0.000000,0.000000,0.000000,0.000000,0.005880,0.533990,0,0,0,0
0,1,32,0.1,1,1.000000e-08,[3],[chr12],0.046988,0.869520,0.000000,0.000000,0.000000,0.000000,0.035934,0.964227,0,0,0,0
0,1,8,1.0,1,1.000000e-08,[3],[chr12],0.102240,0.873784,0.000000,0.000000,0.000000,0.023932,0.008846,0.849095,0,0,0,0
